In [1]:
import math
import numpy as np
import os
import pandas as pd
import pickle
from PIL import Image
from sklearn.metrics import confusion_matrix, matthews_corrcoef
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, GPT2Model, pipeline, AutoModelForCausalLM, \
AutoTokenizer, BitsAndBytesConfig, LlamaTokenizer, LlamaModel, AutoModelForTextEncoding
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from transformers import RobertaModel, RobertaTokenizer


In [15]:
human_bert = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/BERT/human_embeddings.npy', allow_pickle=True)
human_gpt = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/FLAN/human_embeddings.npy', allow_pickle=True)

ai_bert = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/BERT/gpt_embeddings.npy', allow_pickle=True)
ai_gpt = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/FLAN/gpt_embeddings.npy', allow_pickle=True)
ai_embed_length = len(ai_gpt[0])
print(ai_embed_length, len(ai_bert[0][0]))
bert_min_length = min(len(human_bert), len(ai_bert))
human_bert = human_bert[:bert_min_length]
ai_bert = ai_bert[:bert_min_length]

gpt_min_length = min(len(human_gpt), len(ai_gpt))
human_gpt = human_gpt[:bert_min_length]
ai_gpt = ai_gpt[:bert_min_length]

print(len(human_bert), len(human_gpt), len(ai_bert), len(ai_gpt))

human_bert = list(human_bert)
ai_bert = list(ai_bert)
human_gpt = list(human_gpt)
ai_gpt = list(ai_gpt)

human_bert = torch.tensor(human_bert)
ai_bert = torch.tensor(ai_bert)
human_gpt = torch.tensor(human_gpt).reshape([bert_min_length, 1, ai_embed_length])
ai_gpt = torch.tensor(ai_gpt).reshape([bert_min_length, 1, ai_embed_length])

# Display the result tensor


4096 1024
2278 2278 2278 2278


In [16]:
print(human_bert.shape, ai_bert.shape, human_gpt.shape, ai_gpt.shape)  

torch.Size([2278, 1, 1024]) torch.Size([2278, 1, 1024]) torch.Size([2278, 1, 4096]) torch.Size([2278, 1, 4096])


In [45]:
human = torch.cat([human_bert, human_gpt], dim = 2).view(2278, 64, 80)
ai = torch.cat([ai_bert, ai_gpt], dim = 2).view(2278, 64, 80)

In [46]:
embeds = torch.cat((human, ai), dim=0)

print(embeds.shape)

torch.Size([4556, 64, 80])


In [35]:
embeds = embeds.view(4556,64, 80)

In [47]:
print(embeds.shape)

torch.Size([4556, 64, 80])


In [48]:
zeros_tensor = torch.zeros((bert_min_length))
ones_tensor = torch.ones((bert_min_length))

labels = torch.cat((zeros_tensor, ones_tensor), dim=0)

In [49]:
print(len(labels))

4556


In [50]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

test_size = 0.2
dataset = TensorDataset(embeds, labels)

# Split dataset into train and test sets
train_dataset, test_dataset = train_test_split(dataset, test_size=test_size, random_state=42)

# Define batch size
batch_size = 256

# Create DataLoader for training set
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
import torch

import torch.nn as nn
import torch.optim as optim




In [51]:
total_dataset = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [52]:
import torch.nn as nn
import torch.optim as optim

class GPT_CNN2D(nn.Module):
    def __init__(self, embedding_model):
        super(GPT_CNN2D, self).__init__()
        self.em_model = embedding_model
        
            
        if embedding_model == "custom":
            self.conv1 = nn.Conv2d(1, 128, kernel_size=8, padding=1, stride = 2)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.conv2 = nn.Conv2d(128, 64, kernel_size=5, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.conv3 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.flatten = nn.Flatten()            
            # self.fc1 = nn.Linear(96, 2)
            self.fc1 = nn.Linear(384, 2)
            # self.fc2 = nn.Linear(128, 2)
            self.relu = nn.ReLU()

    def forward(self, x):
        
        if self.em_model == "custom":
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.conv3(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.flatten(x)
            # x = self.relu(self.fc1(x))
            x = self.fc1(x)
            # x = self.sigmoid(self.fc2(x))
            return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPT_CNN2D(embedding_model = "custom").to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [53]:
import torch
from transformers import BertModel
# Create an instance of the model
# Map the location to the first CUDA device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
pretrained_weights = torch.load('/home/csgrad/kaushik3/LLM/Code/Embedding_Fusion/Code/gpt_BERT_FLAN.pth', map_location=device)
model.load_state_dict(pretrained_weights)


# Now the model is loaded with the pretrained weights


<All keys matched successfully>

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = GPT_CNN2D(embedding_model = "fusion3").to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
max_val_accuracy = 0
# Step 5: Training Loop
num_epochs = 100

for epoch in range(num_epochs):
    # Validation
    model.eval()
    correct = 0
    total = 0
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    with torch.no_grad():
        for inputs, labels in total_dataset:
            inputs, labels = inputs.to(device), labels.to(device).long()
            inputs = inputs.unsqueeze(1)
            #inputs = inputs.view(1,1,256,64,80)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, 1)
            TP += ((predicted == 1) & (labels == 1)).sum().item()
            FP += ((predicted == 1) & (labels == 0)).sum().item()
            TN += ((predicted == 0) & (labels == 0)).sum().item()
            FN += ((predicted == 0) & (labels == 1)).sum().item()
            total += labels.size(0)
            #print(labels, predicted)
            correct += (predicted == labels).sum().item()
    val_accuracy = correct / total
    if max_val_accuracy<val_accuracy:
        max_TP = TP
        max_FP = FP
        max_TN = TN
        max_FN = FN
    max_val_accuracy = max(max_val_accuracy, val_accuracy)
    
    
    print(f'Validation Accuracy: {val_accuracy:.4f}')
    print(f'TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}')


Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: 60, TN: 2218, FN: 2176
Validation Accuracy: 0.5092
TP: 102, FP: